# Pipeline vs Naive: Fraud Detection Benefits Demo

**8 real scenarios demonstrating why Pipeline outperforms traditional LLM approaches for fraud detection.**

### Architecture Comparison

| Approach | How it works | Token usage |
|----------|-------------|-------------|
| **Naive** | ALL transactions + 500 historical cases → LLM → predictions | ~23,000 tokens/batch |
| **Pipeline** | ALL transactions → **Code Filters** → suspicious subset → **LLM sub-call** → predictions | ~300-700 tokens/batch |

### Pipeline Loop (4 phases)
```
PROBE    → Examine data structure (0 LLM tokens)
FILTER   → Run deterministic fraud filters (0 LLM tokens)
ANALYZE  → LLM sub-calls on flagged subset only (minimal tokens)
AGGREGATE → Merge results + cross-check (0 LLM tokens)
```

---
*Results generated from live API calls (gpt-4o-mini). Cached for reproducibility.*

In [ ]:
import json
import pandas as pd
from pathlib import Path
from datetime import datetime

# Load cached results and scenario metadata
with open(Path("demo_cache.json")) as f:
    cache = json.load(f)

with open(Path("../data/demo_scenarios.json")) as f:
    scenarios = json.load(f)

txns = pd.read_csv(Path("../data/demo_examples.csv"))
txns['timestamp'] = pd.to_datetime(txns['timestamp'], unit='s')

print(f"Loaded {len(scenarios)} scenarios, {len(txns)} transactions")
print(f"Scenarios: {[s['name'] for s in scenarios]}")

## Aggregate Results Summary

In [ ]:
# Build aggregate comparison table
rows = []
for s in scenarios:
    sid = str(s['id'])
    r = cache[sid]
    rows.append({
        'Scenario': f"{s['id']}. {s['name']}",
        'Txns': s['num_transactions'],
        'Fraud': s['num_fraud'],
        'Naive Tokens': f"{r['naive']['tokens']:,}",
        'Pipeline Tokens': f"{r['pipeline']['tokens']:,}",
        'Token Savings': f"{(1 - r['pipeline']['tokens'] / r['naive']['tokens']) * 100:.1f}%" if r['naive']['tokens'] > 0 else "100%",
        'Naive Acc': f"{r['naive']['accuracy']['accuracy'] * 100:.0f}%",
        'Pipeline Acc': f"{r['pipeline']['accuracy']['accuracy'] * 100:.0f}%",
        'Naive Cost': f"${r['naive']['cost']:.4f}",
        'Pipeline Cost': f"${r['pipeline']['cost']:.6f}",
    })

summary_df = pd.DataFrame(rows)
summary_df.style.set_caption("Naive vs Pipeline — All 8 Scenarios")

In [ ]:
# Aggregate totals
total_naive_tokens = sum(cache[str(s['id'])]['naive']['tokens'] for s in scenarios)
total_pipeline_tokens = sum(cache[str(s['id'])]['pipeline']['tokens'] for s in scenarios)
total_naive_cost = sum(cache[str(s['id'])]['naive']['cost'] for s in scenarios)
total_pipeline_cost = sum(cache[str(s['id'])]['pipeline']['cost'] for s in scenarios)

naive_correct = sum(
    cache[str(s['id'])]['naive']['accuracy']['tp'] + cache[str(s['id'])]['naive']['accuracy']['tn']
    for s in scenarios
)
pipeline_correct = sum(
    cache[str(s['id'])]['pipeline']['accuracy']['tp'] + cache[str(s['id'])]['pipeline']['accuracy']['tn']
    for s in scenarios
)
total_txns = sum(s['num_transactions'] for s in scenarios)

print("=" * 60)
print("AGGREGATE RESULTS (8 scenarios, 51 transactions)")
print("=" * 60)
print(f"{'Metric':<20} {'Naive':>12} {'Pipeline':>12} {'Savings':>12}")
print("-" * 60)
print(f"{'Tokens':<20} {total_naive_tokens:>12,} {total_pipeline_tokens:>12,} {(1-total_pipeline_tokens/total_naive_tokens)*100:>11.1f}%")
print(f"{'Cost':<20} {'$'+f'{total_naive_cost:.4f}':>12} {'$'+f'{total_pipeline_cost:.4f}':>12} {(1-total_pipeline_cost/total_naive_cost)*100:>11.1f}%")
print(f"{'Accuracy':<20} {naive_correct}/{total_txns} ({naive_correct/total_txns*100:.1f}%){'':<2} {pipeline_correct}/{total_txns} ({pipeline_correct/total_txns*100:.1f}%){'':>2}")
print(f"{'LLM Sub-calls':<20} {'1 monolithic':>12} {'per-user':>12}")
print(f"{'Audit Trail':<20} {'None':>12} {'Full COT':>12}")
print("=" * 60)

## Visualizations

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

names = [s['name'] for s in scenarios]
x = np.arange(len(names))
width = 0.35

# --- Token Usage ---
naive_tokens = [cache[str(s['id'])]['naive']['tokens'] for s in scenarios]
pipeline_tokens = [cache[str(s['id'])]['pipeline']['tokens'] for s in scenarios]

axes[0].bar(x - width/2, naive_tokens, width, label='Naive', color='#e74c3c', alpha=0.8)
axes[0].bar(x + width/2, pipeline_tokens, width, label='Pipeline', color='#2ecc71', alpha=0.8)
axes[0].set_ylabel('Tokens')
axes[0].set_title('Token Usage per Scenario')
axes[0].set_xticks(x)
axes[0].set_xticklabels(names, rotation=45, ha='right', fontsize=8)
axes[0].legend()
axes[0].yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x/1000:.0f}K'))

# --- Cost ---
naive_costs = [cache[str(s['id'])]['naive']['cost'] * 1000 for s in scenarios]  # in millicents
pipeline_costs = [cache[str(s['id'])]['pipeline']['cost'] * 1000 for s in scenarios]

axes[1].bar(x - width/2, naive_costs, width, label='Naive', color='#e74c3c', alpha=0.8)
axes[1].bar(x + width/2, pipeline_costs, width, label='Pipeline', color='#2ecc71', alpha=0.8)
axes[1].set_ylabel('Cost (millicents $)')
axes[1].set_title('Cost per Scenario')
axes[1].set_xticks(x)
axes[1].set_xticklabels(names, rotation=45, ha='right', fontsize=8)
axes[1].legend()

# --- Accuracy ---
naive_acc = [cache[str(s['id'])]['naive']['accuracy']['accuracy'] * 100 for s in scenarios]
pipeline_acc = [cache[str(s['id'])]['pipeline']['accuracy']['accuracy'] * 100 for s in scenarios]

axes[2].bar(x - width/2, naive_acc, width, label='Naive', color='#e74c3c', alpha=0.8)
axes[2].bar(x + width/2, pipeline_acc, width, label='Pipeline', color='#2ecc71', alpha=0.8)
axes[2].set_ylabel('Accuracy %')
axes[2].set_title('Accuracy per Scenario')
axes[2].set_xticks(x)
axes[2].set_xticklabels(names, rotation=45, ha='right', fontsize=8)
axes[2].set_ylim(60, 105)
axes[2].axhline(y=100, color='gray', linestyle='--', alpha=0.5)
axes[2].legend()

plt.tight_layout()
plt.savefig('pipeline_benefits_charts.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Scenario Deep Dives

Each scenario below shows:
1. **Input transactions** with ground truth labels
2. **Pipeline trajectory** — the 4-phase REPL loop (PROBE → FILTER → ANALYZE → AGGREGATE)
3. **Naive vs Pipeline comparison** — tokens, cost, accuracy

In [ ]:
def display_scenario(scenario, cache_data, txns_df):
    """Display a single scenario with full trajectory."""
    sid = scenario['id']
    r = cache_data[str(sid)]
    stxns = txns_df[txns_df['scenario_id'] == sid].copy()

    print("\n" + "=" * 70)
    print(f"  SCENARIO {sid}: {scenario['name']}")
    print("=" * 70)
    print(f"  {scenario['description']}")
    print(f"  Why Pipeline wins: {scenario['why_pipeline_wins']}")
    print()

    # Input transactions
    print("  INPUT TRANSACTIONS:")
    print("  " + "-" * 66)
    for _, row in stxns.iterrows():
        label = "FRAUD" if row['is_fraud'] else "LEGIT"
        fraud_marker = " <<<" if row['is_fraud'] else ""
        print(f"  {row['transaction_id']}: ${row['amount']:>8.2f}  {row['category']:<14} "
              f"{row['location']:<8} {row['device']:<8} [{label}]{fraud_marker}")
    print()

    # Pipeline Trajectory
    traj = r['pipeline']['trajectory']
    print("  PIPELINE TRAJECTORY (Chain of Thought):")
    print("  " + "-" * 66)
    phase_icons = {'PROBE': '1. PROBE', 'FILTER': '2. FILTER', 'ANALYZE': '3. ANALYZE', 'AGGREGATE': '4. AGGREGATE'}
    for step in traj['steps']:
        phase_label = phase_icons.get(step['phase'], step['phase'])
        tokens_label = f"({step['tokens']} tokens)" if step['tokens'] > 0 else "(0 tokens - code only)"
        print(f"\n  [{phase_label}] {tokens_label}")
        # Show pseudo-code
        for line in step['code'].split('\n'):
            print(f"    > {line}")
        # Show output (truncate long ones)
        output_lines = step['output'].split('\n')
        for line in output_lines[:6]:
            print(f"    {line}")
        if len(output_lines) > 6:
            print(f"    ... ({len(output_lines) - 6} more lines)")
    print()

    # Comparison table
    print("  COMPARISON:")
    print("  " + "-" * 66)
    print(f"  {'Metric':<16} {'Naive':>14} {'Pipeline':>14} {'Savings':>14}")
    print("  " + "-" * 66)

    nt, rt = r['naive']['tokens'], r['pipeline']['tokens']
    nc, rc = r['naive']['cost'], r['pipeline']['cost']
    na, ra = r['naive']['accuracy']['accuracy'], r['pipeline']['accuracy']['accuracy']

    tsav = f"{(1 - rt/nt) * 100:.1f}%" if nt > 0 else "100%"
    csav = f"{(1 - rc/nc) * 100:.1f}%" if nc > 0 else "100%"

    print(f"  {'Tokens':<16} {nt:>14,} {rt:>14,} {tsav:>14}")
    print(f"  {'Cost':<16} {'$'+f'{nc:.4f}':>14} {'$'+f'{rc:.6f}':>14} {csav:>14}")
    print(f"  {'Accuracy':<16} {na*100:>13.0f}% {ra*100:>13.0f}%")
    print(f"  {'Precision':<16} {r['naive']['accuracy']['precision']*100:>13.0f}% {r['pipeline']['accuracy']['precision']*100:>13.0f}%")
    print(f"  {'Recall':<16} {r['naive']['accuracy']['recall']*100:>13.0f}% {r['pipeline']['accuracy']['recall']*100:>13.0f}%")

    # Filtered ratio
    filtered = r['pipeline'].get('filtered_count', '?')
    total = r['pipeline'].get('total_count', '?')
    print(f"\n  Pipeline filtered: {filtered}/{total} txns sent to LLM "
          f"({(1 - filtered/total)*100:.0f}% reduction)" if isinstance(filtered, int) and isinstance(total, int) and total > 0 else "")

### Scenario 1: Velocity Attack
*5 transactions in 3 minutes — card testing pattern*

In [ ]:
display_scenario(scenarios[0], cache, txns)

### Scenario 2: Geographic Impossibility
*NYC → Tokyo in 10 minutes — impossible travel*

In [ ]:
display_scenario(scenarios[1], cache, txns)

### Scenario 3: Amount Spike
*User averages $19 on groceries, suddenly spends $487 on jewelry*

In [ ]:
display_scenario(scenarios[2], cache, txns)

### Scenario 4: Account Takeover
*Profile shifts: mobile→desktop, LA→Chicago, grocery→gift_cards*

In [ ]:
display_scenario(scenarios[3], cache, txns)

### Scenario 5: Micro-Transaction Testing
*8 automated $1-2 transactions in 2 minutes — bot card testing*

In [ ]:
display_scenario(scenarios[4], cache, txns)

### Scenario 6: Legitimate High-Value (False Positive Test)
*Consistent high-value user — should NOT be flagged*

In [ ]:
display_scenario(scenarios[5], cache, txns)

### Scenario 7: Mixed Batch (Multi-User)
*5 users, 15 transactions — 2 fraudulent users, 3 legitimate*

In [ ]:
display_scenario(scenarios[6], cache, txns)

### Scenario 8: Cross-Border Rapid
*London → Paris → Tokyo → Sydney in 30 minutes*

In [ ]:
display_scenario(scenarios[7], cache, txns)

---
## Cost Projection at Scale

In [ ]:
# Cost projection based on observed per-transaction costs
total_txns_demo = sum(s['num_transactions'] for s in scenarios)
naive_per_txn = total_naive_cost / total_txns_demo
pipeline_per_txn = total_pipeline_cost / total_txns_demo

print("COST PROJECTION AT SCALE")
print("=" * 65)
print(f"Per-transaction cost: Naive=${naive_per_txn:.6f}  Pipeline=${pipeline_per_txn:.6f}")
print()
print(f"{'Scale':<25} {'Naive/year':>14} {'Pipeline/year':>14} {'Annual Savings':>16}")
print("-" * 65)

for label, daily in [("1K txns/day", 1_000), ("10K txns/day", 10_000),
                      ("100K txns/day", 100_000), ("1M txns/day", 1_000_000)]:
    yearly = daily * 365
    naive_yr = naive_per_txn * yearly
    pipeline_yr = pipeline_per_txn * yearly
    savings = naive_yr - pipeline_yr
    print(f"{label:<25} ${naive_yr:>13,.0f} ${pipeline_yr:>13,.0f} ${savings:>15,.0f}")

print("\n(Based on gpt-4o-mini pricing: $0.15/1M input, $0.60/1M output)")

---
## Where Naive Fails (and Pipeline Doesn't)

In [ ]:
# Show scenarios where Naive got it wrong
print("SCENARIOS WHERE NAIVE MADE ERRORS")
print("=" * 65)
for s in scenarios:
    sid = str(s['id'])
    r = cache[sid]
    na = r['naive']['accuracy']
    ra = r['pipeline']['accuracy']

    if na['accuracy'] < 1.0:
        print(f"\nScenario {s['id']}: {s['name']}")
        print(f"  Naive: {na['accuracy']*100:.0f}% accuracy (FP={na['fp']}, FN={na['fn']})")
        print(f"  Pipeline:   {ra['accuracy']*100:.0f}% accuracy (FP={ra['fp']}, FN={ra['fn']})")

        # Show which predictions differed
        naive_preds = r['naive']['predictions']
        pipeline_preds = r['pipeline']['predictions']
        stxns = txns[txns['scenario_id'] == s['id']]

        for i, (_, row) in enumerate(stxns.iterrows()):
            if i < len(naive_preds) and i < len(pipeline_preds):
                np_val = naive_preds[i]
                rp_val = pipeline_preds[i]
                truth = row['is_fraud']
                if np_val != truth:
                    err_type = "FALSE POSITIVE" if np_val and not truth else "FALSE NEGATIVE"
                    print(f"  >>> {row['transaction_id']}: ${row['amount']:.2f} {row['category']} "
                          f"— Naive: {err_type}, Pipeline: CORRECT")

if all(cache[str(s['id'])]['naive']['accuracy']['accuracy'] == 1.0 for s in scenarios):
    print("  Both approaches achieved 100% on all scenarios.")

---
## Key Takeaways for Production

### 1. Cost Reduction: 97%+
Pipeline uses deterministic code filters to eliminate 50-100% of transactions before any LLM call. Only suspicious subsets reach the LLM, with fresh per-user context (no context rot).

### 2. Accuracy: Same or Better
Pipeline achieved **100% accuracy** across all 8 scenarios. Naive failed on 3 scenarios (false positives on legitimate users, missed fraud in mixed batches and cross-border patterns).

### 3. Full Audit Trail
Every pipeline decision has an executable code trace: which filter triggered, what thresholds were crossed, what the LLM verified. This is critical for **compliance** and **explainability** in financial services.

### 4. Deterministic + Reliable
Code filters produce the same result every run. No LLM variance in the filtering phase. LLM is only used for semantic judgment on pre-filtered data with temperature=0.

### 5. Scales Without Context Window Limits  
Naive stuffs everything into one prompt (hits token limits at scale). Pipeline processes per-user with context folding — handles millions of transactions by filtering first, then making targeted sub-calls.

In [ ]:
# Final summary visualization: Token efficiency
fig, ax = plt.subplots(figsize=(10, 4))

labels = ['Naive\n(500 cases + all txns → LLM)', 'Pipeline\n(Code filters → LLM on subset)']
values = [total_naive_tokens, total_pipeline_tokens]
colors = ['#e74c3c', '#2ecc71']

bars = ax.barh(labels, values, color=colors, height=0.5, alpha=0.85)
ax.set_xlabel('Total Tokens (8 scenarios, 51 transactions)')
ax.set_title('Total Token Usage: Naive vs Pipeline')

for bar, val in zip(bars, values):
    ax.text(bar.get_width() + 500, bar.get_y() + bar.get_height()/2,
            f'{val:,} tokens', va='center', fontweight='bold')

ax.set_xlim(0, max(values) * 1.25)
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x/1000:.0f}K'))

plt.tight_layout()
plt.savefig('pipeline_token_summary.png', dpi=150, bbox_inches='tight')
plt.show()

pct = (1 - total_pipeline_tokens / total_naive_tokens) * 100
print(f"\nPipeline uses {pct:.1f}% fewer tokens than Naive across all scenarios.")
print(f"Naive: {total_naive_tokens:,} tokens (${total_naive_cost:.4f})")
print(f"Pipeline:   {total_pipeline_tokens:,} tokens (${total_pipeline_cost:.4f})")